In [2]:
import pandas as pd
import uuid

In [3]:
!pip install openpyxl


[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
#Crear dimensiones base para variable a nivel sujeto
temaInteres = pd.DataFrame(columns=['id','nombre','descripcion'])

etapa= pd.DataFrame(columns=['id','numEtapa','nombreAnalisis','nombreBD','descripcion',
                             'eventoInicial','eventoFinal'])

fecha=pd.DataFrame(columns=['dia','mes','anio'])

evento=pd.DataFrame(columns=['id','nombre','descripcion'])

variable=pd.DataFrame(columns=['id','nombreBD','nombreAnalisis','unidades','edadCorregida','derivada',
                               'impacto','tipoDato','tipoVariable','nivelMedicion','longitudinal',
                               'descripcionCorta','descripcionLarga'])


In [5]:
# Conseguir dataframes de diccionario
file_dict=pd.ExcelFile('data/KMC-diccionario.xlsx')
df_etapas=pd.read_excel(file_dict, sheet_name='ETAPAS')
df_temas=pd.read_excel(file_dict, sheet_name='Temas de Interés')
df_variables=pd.read_excel(file_dict, sheet_name='KMC-12mEC-DIC-VAR-version2024')

In [6]:
#Poblar dimensión de eventos
eventos = pd.unique(
    pd.concat([
        df_etapas['Evento inicial (id-var)'],
        df_etapas['Evento-final (id-var)']
    ])
).tolist()

ids_eventos = [str(uuid.uuid4()) for _ in range(len(eventos))]

for i in range(len(eventos)):
    evento.loc[len(evento)] = [
        ids_eventos[i],
        eventos[i],
        ""
    ]

In [7]:
#Poblar dimensión de etapas - sin eventos
def encontrar_evento(nombre):
    fila=evento[evento['nombre']==nombre]
    return fila['id'].values[0]

ids_etapas = [str(uuid.uuid4()) for _ in range(len(df_etapas))]

for index, row in df_etapas.iterrows():
    etapa.loc[row['NUMERO ETAPA']]=[
        ids_etapas[index],
        row['NUMERO ETAPA'],
        row['ID-ETAPA'].lower(),
        row['NOMBRE CORTO'].lower(),
        row['DESCRIPCION'].lower(),
        encontrar_evento(row['Evento inicial (id-var)']),
        encontrar_evento(row['Evento-final (id-var)'])
    ]

In [ ]:
#Poblar dimensión tema de interes
ids_temas = [str(uuid.uuid4()) for _ in range(len(df_temas))]

for index,row in df_temas.iterrows():
    temaInteres.loc[row['Número TdI']]=[
        ids_temas[index],
        row['ID-TdI'],
        row['DescripciOn']
    ]

,id,nombre,descripcion
0,2a5ab9c5-f414-46ef-9cd9-48d67214c382,CRECIMIENTO,"datos de peso- talla-pc, cada vez que sea posi..."
1,122c9370-b27f-471d-bc6a-e6361c75ed62,CONTEXTO SOCIOECONOMICO,NaN
2,4f600fb7-c7a7-4a6a-87bc-562d6bcb4364,DESARROLLO NEUROMOTOR,anoxia5mn
3,072809fa-9c51-4ee0-91df-9571b989b2ea,DESARROLLO CRECIMIENTO,NaN
4,41aa4de4-0151-46d1-8762-9bed55e279c9,DESARROLLO SENSORIAL,NaN
5,6813537b-58de-49b1-82c3-2fcf8906e4cb,NUTRICION,NaN
6,53f78c0e-65b6-4d11-bb0c-318489098489,APEGO(?),NaN
7,7b0f3cdf-d9dc-4335-bb3e-f077f2d0f351,HOSPITALIZACIONES,NaN
8,c8c94fc3-c0c5-4570-a2f2-8875260e52c1,SEGUIMIENTO-VACUNAS-TAMIZ-MORT,"Seguimiento en vacunas, tamizados,"
9,621616b2-62f2-411f-9908-8f93aa7c7e38,POSICIÓN CANGURO,NaN


In [ ]:
#Poblar dimension fecha
fecha.loc[0]=[
    1,
    1,
    2025
]

,dia,mes,anio
0,1,1,2025
